In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx, Ly = (75, 75)
nx, ny = (460, 460)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]


problem = de.IVP(domain, variables=['rho1','rho2','rho3','rho4','rhom'])
problem.parameters['D0'] = Diff
problem.parameters['K'] = K
problem.parameters['g0'] = g


problem.substitutions['rho_b'] = "rho1+rho2+rho3+rho4+rhom" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem.add_equation("dt(rho1)  - D0*Lap(rho1)  = g0*grow(rho1,rho_b) - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D0*Lap(rho2)  = g0*grow(rho2,rho_b) - K*rho2*rho1")
problem.add_equation("dt(rho3)  - D0*Lap(rho3)  = g0*grow(rho3,rho_b) - K*rho3*rho4")
problem.add_equation("dt(rho4)  - D0*Lap(rho4)  = g0*grow(rho4,rho_b) - K*rho4*rho3")

problem.add_equation("dt(rhom)  = 2*K*(rho1*rho2 + rho3*rho4 )")



In [ ]:
ts = de.timesteppers.RK443

In [ ]:
solver =  problem.build_solver(ts)

In [ ]:
np.random.seed(4)
p = 60*np.random.rand(20,2)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']
rho3 = solver.state['rho3']
rho4 = solver.state['rho4']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

r0 = 0.75
w = 0.22

offx = 5
offy = 5

def blob_(x0,y0):
    return 0.5*blob(x,y,x0+offx,y0+offy,r0,w)
    
rho1['g'] = blob_(p[0][0] ,p[0][1]) +blob_(p[1][0] ,p[1][1]) +blob_(p[2][0] ,p[2][1]) + blob_(p[4][0] ,p[4][1]) + blob_(p[8][0] ,p[8][1])+ blob_(p[10][0] ,p[10][1])+ blob_(p[11][0] ,p[11][1])+ blob_(p[13][0] ,p[13][1])+ blob_(p[19][0] ,p[19][1])
rho2['g'] = blob_(p[3][0] ,p[3][1]) + blob_(p[5][0] ,p[5][1])+ blob_(p[6][0] ,p[6][1])+ blob_(p[7][0] ,p[7][1])+ blob_(p[9][0] ,p[9][1])+ blob_(p[12][0] ,p[12][1])+ blob_(p[14][0] ,p[14][1])+ blob_(p[15][0] ,p[15][1])+ blob_(p[16][0] ,p[16][1])+ blob_(p[17][0] ,p[17][1])+ blob_(p[18][0] ,p[18][1])
rho3['g'] = blob_(p[1][0] ,p[1][1]) + blob_(p[5][0] ,p[5][1])+ blob_(p[6][0] ,p[6][1]) + blob_(p[7][0] ,p[7][1])+ blob_(p[8][0] ,p[8][1])+ blob_(p[16][0] ,p[16][1])+ blob_(p[18][0] ,p[18][1])+ blob_(p[19][0] ,p[19][1])+ blob_(p[13][0] ,p[13][1])
rho4['g'] = blob_(p[0][0] ,p[0][1]) +blob_(p[2][0] ,p[2][1]) + blob_(p[3][0] ,p[3][1]) + blob_(p[4][0] ,p[4][1])+ blob_(p[9][0] ,p[9][1])+ blob_(p[10][0] ,p[10][1])+ blob_(p[11][0] ,p[11][1])+ blob_(p[12][0] ,p[12][1])+ blob_(p[14][0] ,p[14][1])+ blob_(p[15][0] ,p[15][1])+ blob_(p[17][0] ,p[17][1])

In [ ]:
solver.stop_sim_time = 440
dt = 0.1*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('voronoi', sim_dt=3, max_writes=900)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho3')
analysis.add_task('rho4')
analysis.add_task('rhom')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
from dedalus.tools import post
post.merge_process_files("voronoi", cleanup=True)